# Todo list

1. one_hot √

* mini-batch

* normalization

* train/dev/test set √

* linear function √

* sigmoid function

* tanh function

* relu function √

* softmax function √

* loss function √

* cost function

* regularization

* drop out

* batch normalization

* momentum

* exponentially moving average

* Adam

* all backpropagation of above

In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
from pprint import pprint
""" helper functions """
def names_in(dictionary):
    """ list all names in a dictionary """
    print([name for name,_ in sorted(dictionary.items())])
def names_shape_in(dictionary):
    pprint([(name, val.shape) for name,val in sorted(dictionary.items())])

def debug_show_all_variables():
    print("cache: ", end='')
    names_shape_in(cache)
    print("parameters: ", end='')
    names_shape_in(parameters)
    print("hyper_parameters: ", end='')
    names_in(hyper_parameters)



In [3]:
def load_mnist():
    import struct
    from array import array
    """ 
    load MNIST dataset into numpy array 
    MNIST dataset can be downloaded manually.
    url: http://yann.lecun.com/exdb/mnist/
    """
    ret = {}
    with open('MNIST/train-images.idx3-ubyte', 'rb') as f:
        magic, size, rows, cols = struct.unpack(">IIII", f.read(16))
        assert(magic==2051)
        ret['X_train'] = np.array(array("B", f.read())).reshape(size,rows,cols)

    with open('MNIST/t10k-images.idx3-ubyte', 'rb') as f:
        magic, size, rows, cols = struct.unpack(">IIII", f.read(16))
        assert(magic==2051)
        ret['X_test'] = np.array(array("B", f.read())).reshape(size,rows,cols)

    with open('MNIST/train-labels.idx1-ubyte', 'rb') as f:
        magic, size = struct.unpack(">II", f.read(8))
        assert(magic==2049)
        ret['Y_train'] = np.array(array("B", f.read())).reshape(size,1)

    with open('MNIST/t10k-labels.idx1-ubyte', 'rb') as f:
        magic, size = struct.unpack(">II", f.read(8))
        assert(magic==2049)
        ret['Y_test'] = np.array(array("B", f.read())).reshape(size,1)

    return ret

In [4]:
mnist_original = load_mnist()

""" random shuffle the training set """
np.random.seed(1)
permutation = np.random.permutation(mnist_original['X_train'].shape[0])
mnist_original['X_train'] = mnist_original['X_train'][permutation]
mnist_original['Y_train'] = mnist_original['Y_train'][permutation]

""" divide trainset into trainset and devset """
len_of_dev = 10000
mnist_original['X_dev'] = mnist_original['X_train'][:len_of_dev]
mnist_original['Y_dev'] = mnist_original['Y_train'][:len_of_dev]
mnist_original['X_train'] = mnist_original['X_train'][len_of_dev:]
mnist_original['Y_train'] = mnist_original['Y_train'][len_of_dev:]

print('X_train:', mnist_original['X_train'].shape,
      'X_dev:', mnist_original['X_dev'].shape,
      'X_test:', mnist_original['X_test'].shape)
print('Y_train:', mnist_original['Y_train'].shape,
      'Y_dev:', mnist_original['Y_dev'].shape,
      'Y_test:', mnist_original['Y_test'].shape)


X_train: (50000, 28, 28) X_dev: (10000, 28, 28) X_test: (10000, 28, 28)
Y_train: (50000, 1) Y_dev: (10000, 1) Y_test: (10000, 1)


In [5]:
def manually_validate_dataset(dataset):
    random_train = np.random.randint(1, len(dataset['X_train']))-1
    random_dev = np.random.randint(1, len(dataset['X_dev']))-1
    random_test = np.random.randint(1, len(dataset['X_test']))-1
    print(dataset['Y_train'][random_train], dataset['Y_dev'][random_dev], dataset['Y_test'][random_test])
    fig, (ax1, ax2, ax3) = plt.subplots(1,3, sharey=True, figsize=[10,3])
    ax1.imshow(dataset['X_train'][random_train], cmap='gray')
    ax2.imshow(dataset['X_dev'][random_dev], cmap='gray')
    ax3.imshow(dataset['X_test'][random_test], cmap='gray')
    plt.show()

#manually_validate_dataset(mnist_original)

$$ X = 
\begin{bmatrix}
\vert & & \vert & & \vert \\
x^{(1)} & ... & x^{(i)} & ... & x^{(m)} \\
\vert & & \vert & & \vert
\end{bmatrix} 
\quad \quad
Y = 
\begin{bmatrix}
\vert & & \vert & & \vert \\
y_{one\_hot}^{(1)} & ... & y_{one\_hot}^{(i)} & ... & y_{one\_hot}^{(m)} \\
\vert & & \vert & & \vert \\
\end{bmatrix}
$$

In [6]:
mnist = {}

""" X is 28*28 image """
def flatten(X):
    """ prepare X to (nx, m) shape """
    X = X.reshape(-1, 28*28).T
    return X

mnist['X_train'] = flatten(mnist_original['X_train'])
mnist['X_dev'] = flatten(mnist_original['X_dev'])
mnist['X_test'] = flatten(mnist_original['X_test'])

""" Y is label 0-9 """
def one_hot(Y, C):
    """ prepare Y to (1, m) shape """
    assert(Y.shape[1]==1)
    Y_ret = np.zeros((Y.shape[0], C))
    Y_ret[np.arange(Y.shape[0]), Y.reshape(-1).astype(int)] = 1
    Y_ret = Y_ret.T
    return Y_ret

def test_one_hot():
    Y = np.ones((5,1))
    Y = one_hot(Y, 10)
    assert(Y[0,0]==0)
    assert(Y[1,0]==1)
    assert(Y[2,1]==0)

def back_one_hot(Y):
    """ convert one hot Y back to real number """
    Y_ret = np.repeat( [np.arange(Y.shape[0])], repeats=Y.shape[1], axis=0 )
    assert(Y_ret.shape == Y.T.shape)
    Y_ret = Y_ret[Y.T.astype(bool)]
    return Y_ret.reshape(-1,1)

mnist['Y_train'] = one_hot(mnist_original['Y_train'], 10)
mnist['Y_dev'] = one_hot(mnist_original['Y_dev'], 10)
mnist['Y_test'] = one_hot(mnist_original['Y_test'], 10)

print(mnist['X_train'].shape, mnist['Y_train'].shape)

(784, 50000) (10, 50000)


In [7]:
""" init cache, parameters and hyper_parameters """
def init(X_size, hidden_layers=[10,10,5], C=10, learning_rate=0.01):
    global cache, parameters, hyper_parameters
    cache = {}
    parameters = {}
    hyper_parameters = {}
    initialize_hyper_parameters(X_size, hidden_layers, C, learning_rate)
    initialize_parameters()

def initialize_hyper_parameters(X_size, hidden_layers, C, learning_rate):
    global hyper_parameters
    hyper_parameters['X_size'] = X_size
    # layers of network, include the last softmax layer
    # hidden layers use ReLU activation function, and the last layer use Softmax function
    hyper_parameters['layers'] = hidden_layers + [C]
    # L layers in total
    hyper_parameters['L'] = len(hyper_parameters['layers'])
    # Class numbers 0-9 is 10
    hyper_parameters['C'] = C
    hyper_parameters['learning_rate'] = learning_rate

def initialize_parameters():
    """ init W b or any other parameters in every layer """
    global parameters, hyper_parameters
    layers = hyper_parameters['layers']
    x_size = hyper_parameters['X_size']
    cells_prev = x_size
    for layer_idx, cells in enumerate(layers):
        parameters['W'+str(layer_idx+1)] = np.random.randn(cells, cells_prev) * 0.01
        parameters['b'+str(layer_idx+1)] = np.zeros((cells, 1))
        cells_prev = layers[layer_idx]

def test_init():
    init(50)
    debug_show_all_variables()
#test_init()

$$
ReLu = max(0, x)
\quad \quad
Softmax = \frac{\exp(Z)}{\sum_i^n{\exp(Z)}}
$$

$$
Z = W \dot X + b
\quad \quad
A = active(Z)
$$

In [8]:
def ReLU(X):
    return X * (X > 0)

def test_ReLU():
    X = np.array([[1., 2., -2., -3.], [-2, -2, 1, 0.1]])
    Y = np.array([[1., 2., 0., 0.], [0, 0, 1, 0.1]])
    bias = np.sum(np.abs(Y - ReLU(X)))
    assert(bias<0.0001)
#test_ReLU()

def softmax(X):
    s = np.sum(np.exp(X), axis=0)
    return np.exp(X) / s

def test_softmax():
    X = np.array([-3.44,1.16,-0.81,3.91])
    Y = np.array([0.0006, 0.0596, 0.0083, 0.9315])
    bias = np.sum(np.abs(Y - softmax(X)))
    assert(bias<0.0001)
    X = np.array([[1.,2,3,4],[2,2,3,4],[5,5,5,4]])
    Y = np.array([[ 0.01714783,  0.0452785 ,  0.10650698,  0.33333333],
       [ 0.04661262,  0.0452785 ,  0.10650698,  0.33333333],
       [ 0.93623955,  0.909443  ,  0.78698604,  0.33333333]])
    bias = np.sum(np.abs(Y - softmax(X)))
    assert(bias<1e-5)
#test_softmax()

def forward_propagation(X):
    global cache, parameters, hyper_parameters
    layers = hyper_parameters['layers']
    A = X
    for layer_idx, cells in enumerate(layers):
        Z = np.dot(parameters['W'+str(layer_idx+1)], A)
        if layer_idx<len(layers)-1:
            """ normal layers use relu """
            A = ReLU(Z)
        else:
            """ last layers use softmax """
            A = softmax(Z)
        cache['Z'+str(layer_idx+1)] = Z
        cache['A'+str(layer_idx+1)] = A
    return A

$$
L(\hat{y}, y) = -\frac{1}{m} \sum_i^m{(y_i\log(\hat{y}_i) + (1-y_i)\log(1-\hat{y}_i))}
$$

$$
L(\hat{y}, y) = -\frac{1}{m} \sum_i^m \sum_j^C (y_j^{(i)}\log(\hat{y}_j^{(i)}))
$$

where m is batch size, C is class number (10 classes), (i) is the i-th example, j is the vertical bit of result.

In [9]:
def loss(Y_hat, Y):
    m = Y.shape[1]
    A = np.multiply(Y, np.log(Y_hat))
    C = -np.sum(A) / m
    return C

def test_loss():
    Y = np.asarray([[0., 1., 1.], [1., 0., 0.]])
    aL = np.array([[.8, .9, .4], [.2, .1, .6]])
    assert(loss(aL, Y) - 0.87702971 < 1e-7)
    
    z = np.array([[1.,2,3,4],[2,2,3,4],[5,5,5,4]])
    y = np.array([[0.,0,1],[0,0,1],[0,0,1],[1,0,0]]).T
    bias = 0.37474097876709611 - loss(softmax(z), y)
    assert(np.abs(bias)<1e-5)
#test_loss()

In [10]:
def predict(Y_hat):
    return np.argmax(Y_hat,axis=0).reshape(-1,1)

def test_predict():
    Y = np.array([[0.9,0.01,0.01,0.01,0.02,0.01,0.01,0.01,0.01,0.01],
                  [0.01,0.01,0.01,0.02,0.9,0.01,0.01,0.01,0.01,0.01]
                 ]).T
    print(Y.shape)
    Y_result = predict(Y)
    print(Y_result)
    Z = np.array([0,4]).reshape(2,1)
    assert(np.sum(Y_result != Z)==0)
#test_predict()

def accuracy(Y_predict, Y):
    return np.sum(np.equal(Y_predict, back_one_hot(Y))) / Y_predict.shape[0]

We already know L formula above:
$$
L = -\frac{1}{m} \sum_i^{m}[y^{(i)} log(\hat{y}^{(i)}) + (1-y^{(i)}) log(1-\hat{y}^{(i)})]
$$

$$
A^{[L]} = \hat{Y} = 
\begin{bmatrix}
\hat{y}^{(1)} & ... & \hat{y}^{(m)} 
\end{bmatrix}
\quad, \quad
Y =
\begin{bmatrix}
y^{(1)} & ... & y^{(m)}
\end{bmatrix}
$$

So:
$$
"dAL" = \frac{\partial L}{\partial A^{[L]}}
= - \frac{1}{m} ( \frac{Y}{A^{[L]}} - \frac{1-Y}{1-A^{[L]}} )
= \frac{A^{[L]}-Y}{m A^{[L]}(1-A^{[L]})}
$$

In [11]:
def backpropagate_cost(Y, AL):
    #m = Y.shape[1] #why not 1/m?
    m=1
    return -1/m * (np.divide(Y, AL) - np.divide(1-Y,1-AL))

def test_backpropagation_cost(epslon=1e-7):
    """ TODO: implement the gradient check """
#test_backpropagation_cost()

$$
"dZL"
= \frac{\partial L}{\partial Z^{[L]}}
= \frac{\partial L}{\partial A^{[L]}} \frac{\partial A^{[L]}}{\partial Z^{[L]}}
= "dAL" * \frac{\partial A^{[L]}}{\partial Z^{[L]}}
$$
$$
\frac{\partial A^{[L]}}{\partial Z^{[L]}}
= A^{[L]} - (A^{[L]})^{2}
$$
$$
"dZL" = "dAL" * ("AL" - "AL"^{2})
$$

In [12]:
def backpropagate_softmax(AL, dAL):
    return np.multiply(dAL, (AL-np.power(AL,2)))

def test_backpropagation_softmax():
    z = np.array([[1.,2,3,4],[2,2,3,4],[5,5,5,4]])
    y = np.array([[0.,0,1],[0,0,1],[0,0,1],[1,0,0]]).T
    a = softmax(z)
    l = loss(a, y)
    da = backpropagate_cost(y, a)
    dz = backpropagate_softmax(a, da)
    result = np.array([[ 0.01714783,0.0452785,0.10650698,-0.66666667],
                     [ 0.04661262,0.0452785,0.10650698,0.33333333],
                     [-0.06376045,-0.090557,-0.21301396,0.33333333]])
    assert(np.sum(np.abs(result - dz))<1e-6)
#test_backpropagation_softmax()

In [13]:
def backpropagate_linear(dZ, W, A_prev):
    m = dZ.shape[1]
    dA = np.dot(W.T, dZ)
    dW = 1/m * np.dot(dZ, A_prev.T)
    db = 1/m * np.sum(dZ, axis=1).reshape(-1,1)
    return dA, dW, db

def backpropagate_ReLU(dA, Z):
    return np.multiply(dA, (Z>=0).astype(np.float32))

def test_backpropagation_linear():
    """ TODO """
    A_prev = np.array([[1,2],[2,2],[3,2]])
    W = 
    pass

def test_backpropagation_ReLU():
    """ TODO """
    pass

In [14]:
def backpropagate_all():
    global mnist, cache, parameters, hyper_parameters
    # 1. Last layer has coss function and softmax function
    L = str(hyper_parameters['L'])
    cache['dA'+L] = backpropagate_cost(mnist['Y_train'],cache['A'+L])
    AL = cache['A'+L]
    dAL = cache['dA'+L]
    cache['dZ'+L] = backpropagate_softmax(AL, dAL)

    # 2. Layers in between are similar: linear and ReLU
    for i in np.arange(start=hyper_parameters['L']-1, stop=0, step=-1):
        L = str(i)
        cache['dA'+L], cache['dW'+str(i+1)], cache['db'+str(i+1)] = backpropagate_linear(cache['dZ'+str(i+1)], 
                                                                           parameters['W'+str(i+1)], 
                                                                           cache['A'+L])
        cache['dZ'+L] = backpropagate_ReLU(cache['dA'+L], cache['Z'+L])

    # 3. First layer has different parameter.
    _, cache['dW1'], cache['db1'] = backpropagate_linear(cache['dZ1'], parameters['W1'], mnist['X_train'])

In [15]:
def update_parameters():
    global cache, parameters
    for i in range(hyper_parameters['L']):
        L = str(i+1)
        parameters['W'+L] -= cache['dW'+L] * hyper_parameters['learning_rate']
        parameters['b'+L] -= cache['db'+L] * hyper_parameters['learning_rate']

In [16]:
def model(X, Y, learning_rate=0.01):
    init(hidden_layers=[100], C=Y.shape[0], X_size=X.shape[0], learning_rate=learning_rate)
    for i in range(100):
        Y_hat = forward_propagation(X)
        cost = loss(Y_hat, Y)
        if i%1==0:
            Y_predict = predict(Y_hat)
            accu = accuracy(Y_predict, Y)
            print(i,'>',cost,'; accu:',accu)
        backpropagate_all()
        update_parameters()
    
model(mnist['X_train'], mnist['Y_train'])

0 > 3.60142930093 ; accu: 0.10662
1 > 10.1497381687 ; accu: 0.20242
2 > 22.6295464351 ; accu: 0.1757
3 > 11.1491189949 ; accu: 0.22128
4 > 4.49205669956 ; accu: 0.20412
5 > 2.26513506491 ; accu: 0.16568
6 > 2.16754719378 ; accu: 0.23076
7 > 2.0562889961 ; accu: 0.30354
8 > 1.97480817826 ; accu: 0.28832
9 > 1.8522214557 ; accu: 0.30446
10 > 1.77600917343 ; accu: 0.3342
11 > 3.19505777056 ; accu: 0.29446
12 > 5.22995157641 ; accu: 0.1263
13 > 6.18417790274 ; accu: 0.20616
14 > 2.20976338922 ; accu: 0.20908
15 > 2.05161577518 ; accu: 0.31778
16 > 2.11779477663 ; accu: 0.26952
17 > 1.937347358 ; accu: 0.29524
18 > 2.05581713001 ; accu: 0.2624
19 > 1.9971302847 ; accu: 0.246
20 > 2.01954391376 ; accu: 0.25066
21 > 1.83482643227 ; accu: 0.31578
22 > 1.9195396734 ; accu: 0.27142
23 > 1.76081143961 ; accu: 0.31652
24 > 1.71360316115 ; accu: 0.36064
25 > 1.5566262188 ; accu: 0.39964
26 > 2.21935654167 ; accu: 0.39012
27 > 3.68112808289 ; accu: 0.32994
28 > 4.05900419912 ; accu: 0.26832


/home/liusida/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:4: RuntimeWarning: invalid value encountered in true_divide
/home/liusida/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:2: RuntimeWarning: invalid value encountered in greater
  from ipykernel import kernelapp as app
/home/liusida/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:9: RuntimeWarning: invalid value encountered in greater_equal


29 > nan ; accu: 0.09942
30 > nan ; accu: 0.09942
31 > nan ; accu: 0.09942
32 > nan ; accu: 0.09942
33 > nan ; accu: 0.09942
34 > nan ; accu: 0.09942
35 > nan ; accu: 0.09942
36 > nan ; accu: 0.09942
37 > nan ; accu: 0.09942
38 > nan ; accu: 0.09942
39 > nan ; accu: 0.09942
40 > nan ; accu: 0.09942
41 > nan ; accu: 0.09942
42 > nan ; accu: 0.09942
43 > nan ; accu: 0.09942
44 > nan ; accu: 0.09942
45 > nan ; accu: 0.09942
46 > nan ; accu: 0.09942
47 > nan ; accu: 0.09942
48 > nan ; accu: 0.09942
49 > nan ; accu: 0.09942
50 > nan ; accu: 0.09942
51 > nan ; accu: 0.09942
52 > nan ; accu: 0.09942
53 > nan ; accu: 0.09942
54 > nan ; accu: 0.09942
55 > nan ; accu: 0.09942
56 > nan ; accu: 0.09942
57 > nan ; accu: 0.09942
58 > nan ; accu: 0.09942
59 > nan ; accu: 0.09942
60 > nan ; accu: 0.09942
61 > nan ; accu: 0.09942
62 > nan ; accu: 0.09942
63 > nan ; accu: 0.09942
64 > nan ; accu: 0.09942
65 > nan ; accu: 0.09942
66 > nan ; accu: 0.09942
67 > nan ; accu: 0.09942
68 > nan ; accu: 0.09942


In [17]:
debug_show_all_variables()

cache: [('A1', (100, 50000)),
 ('A2', (10, 50000)),
 ('Z1', (100, 50000)),
 ('Z2', (10, 50000)),
 ('dA1', (100, 50000)),
 ('dA2', (10, 50000)),
 ('dW1', (100, 784)),
 ('dW2', (10, 100)),
 ('dZ1', (100, 50000)),
 ('dZ2', (10, 50000)),
 ('db1', (100, 1)),
 ('db2', (10, 1))]
parameters: [('W1', (100, 784)), ('W2', (10, 100)), ('b1', (100, 1)), ('b2', (10, 1))]
hyper_parameters: ['C', 'L', 'X_size', 'layers', 'learning_rate']
